# Example of using the orm classes

Imports

In [1]:
from core.orm import Borehole, Interval, Component, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
! rm ./tmp/test_orm_db.db

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-11 15:57:05,143 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-11 15:57:05,144 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 15:57:05,148 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-11 15:57:05,149 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 15:57:05,152 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-11 15:57:05,153 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 15:57:05,155 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-11 15:57:05,156 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 15:57:05,157 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-11 15:57:05,157 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 15:57:05,158 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-11 15:57:05,159 INFO sqlalchemy.engine.base.Engine ()
2020-12-11

Create the objects

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':'../data/test.las', 'F02':'../data/test.las'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
boreholes = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    # print(d)
    boreholes[bh_id].intervals_values = d
    bh_id += 1         

In [8]:
class Project:
    def __init__(self, session):
        self.session = session
        self.refresh()
        #self.boreholes = None

    def refresh(self):
        self.boreholes = self.session.query(Borehole).all()

        
    def commit(self):
        self.session.commit()
        
    def add_borehole(self, bh):
        self.session.add(bh)
        self.commit()
        self.refresh()

    def row_count(self):
        return len(self.boreholes)

    def write_db(self):
        pass

Create the session

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [10]:
p = Project(session)

2020-12-11 15:57:05,323 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 15:57:05,325 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-11 15:57:05,327 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [11]:
for bh in boreholes:
    p.add_borehole(bh)

2020-12-11 15:57:05,347 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-11 15:57:05,350 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 15:57:05,358 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description) VALUES (?, ?, ?, ?)
2020-12-11 15:57:05,358 INFO sqlalchemy.engine.base.Engine ((0, 'F01', 0, 'Anhydrite'), (1, 'F01', 1, 'Sandstone, grey, vf-f'), (2, 'F01', 2, 'Anhydrite'), (3, 'F01', 3, 'Dolomite'), (4, 'F01', 4, 'Anhydrite'), (5, 'F01', 5, 'Sandstone, grey, vf-f'), (6, 'F01', 6, 'Siltstone, red'), (7, 'F01', 7, 'Dolomite')  ... displaying 10 of 25 total bound parameter sets ...  (23, 'F01', 23, 'Limestone'), (24, 'F01', 24, 'Volcanic'))
2020-12-11 15:57:05,361 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-11 15:57:05,366 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 15:57:05,367 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"


In [12]:
p.boreholes[0].id

'F01'

In [13]:
p.boreholes[1].id='F33'

In [14]:
p.boreholes[1].id

'F33'

In [15]:
p.commit()

2020-12-11 15:57:05,435 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET id=? WHERE "Boreholes".id = ?
2020-12-11 15:57:05,437 INFO sqlalchemy.engine.base.Engine ('F33', 'F02')
2020-12-11 15:57:05,443 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
p.boreholes[0].intervals[0].description

2020-12-11 15:57:05,471 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 15:57:05,474 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-11 15:57:05,475 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 15:57:05,489 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-11 15:57:05,491 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [17]:
session.close()

2020-12-11 15:57:05,502 INFO sqlalchemy.engine.base.Engine ROLLBACK
